<h1 style="text-align: center;">Machine Learning Exercise 5 - Hyper Parameter Tuning</h1>
<h2 style="text-align: center;">Ajay Badrinath</h2>
<h3 style="text-align: center;">21011102020</h3>

### Hyper Parameter Tuning 


In [19]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer as s
from sklearn.impute import KNNImputer as knn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [3]:
data=pd.read_csv(r"D:\\tcc.csv")


In [4]:
class Utils_Suite():
    def __init__(self,data):
        self.data=data
    def compute_correlation(self,threshold=0.3):
        matrix=self.data.corr(numeric_only=False)
        x=matrix[(matrix["Churn"]<threshold)&(matrix["Churn"]>-threshold)]["Churn"]
        return x
    def compute_mutual_information(self,thresh=0.1):
        enc = OrdinalEncoder()
        df_encoded = enc.fit_transform(self.data)
        mi_scores = mutual_info_regression(df_encoded, self.data['Churn'])
        mi_scores_df = pd.DataFrame(mi_scores, index=self.data.columns, columns=['Score'])
        return mi_scores_df[mi_scores_df['Score']<thresh]
    def compute_vif(self):
        x=self.data.iloc[:,:-1]
        y=self.data.iloc[:,-1]
        x=pd.DataFrame(x)

        x['intercept']=1
        vif=pd.DataFrame()
        vif['variable']=x.columns
        vif['vif']=[variance_inflation_factor(x.values,i)for i in range(x.shape[1])]
        return vif

In [5]:
class PreProcess():
    #Auto Run Upon Initiation .
    def __init__(self,data):
        self.data=data
        
        self.run()
    # PreProcessing  Schedules
    def run(self):
        self.ClearNull(threshold=0.5)
        self.data=self.data.drop(data[data['TotalCharges'] == ' '].index)
        self.data['TotalCharges']=self.data['TotalCharges'].astype(float)
        l=self.get_all_Null(dtype='float64')
        
        self.outlier_remove('TotalCharges')
        self.outlier_remove('MonthlyCharges')
        
        self.StdScale()
        
        self.drop_uniq_thresh(thresh=5)
        
        #self.Label_Encoding()
        self.one_hot_encoding()
        #self.drop_correlation()
        #self.drop_vif(thresh=4)

    # Remove Correlation 
    def drop_correlation(self):
        k=Utils_Suite(self.data).compute_correlation(0.3)
        f=pd.DataFrame(k)
        m=list(f[(f['Churn']<0.15) & (f['Churn']>-0.15)].index)
        self.data=self.data.drop(columns=m)

    def ClearNull(self,threshold):
        x=self.data.isna().sum()>0

        for i in  list(x.index):
            thresh=self.data[i].isna().sum()/len(self.data)
            if(x[i]==True and thresh>threshold):
                print(i,self.data[i].isna().sum())
                self.data=self.data.drop(i,axis=1)


    def knn_impute(self,n_neighbors,col_list):
        imputer=knn(n_neighbors=n_neighbors)
        for i in col_list:
            self.data[i]=imputer.fit_transform(self.data[[i]])[0][0]


    def get_all_Null(self,dtype=""):
        x=self.data.isna().sum()>0
        l=[]
        for i in  list(x.index):
            thresh=self.data[i].isna().sum()/len(self.data)
            if(x[i]==True and (data[i].dtypes==dtype) ):
                print(i,data[i].isna().sum())
                l+=[i]
        return l
    # Drop Outlier Rows 
    def outlier_remove(self,col):
        
        q1=self.data[col].quantile(0.25)
        q3=self.data[col].quantile(0.75)
        iqr=q3-q1
        l_whis=q1-1.5*iqr
        u_whis=q3+1.5*iqr
        self.data= self.data[(self.data[col]>=l_whis)& (self.data[col]<=u_whis)]
    
    # One hot Encoding using get_dummies
    def one_hot_encoding(self):
        z=(self.data.dtypes=='object')
        k=pd.DataFrame(z)
        obj_list=list(k[k[0]==True].index)
        print(obj_list)
        for i in obj_list:
            dummy=pd.get_dummies(self.data[i],prefix=i,drop_first=True)
            
            self.data=self.data.drop(i,axis=1)
            self.data=self.data.join(dummy)
            
    # standardize data
    def drop_uniq_thresh(self,thresh=5):
        col=data.columns
        x=pd.DataFrame(self.data.dtypes)
        ll=list(x[x[0]=="object"].index)
        droplist=[]
        for i in ll:
            print(i)
            if (len(self.data[i].unique())>thresh):
                droplist+=[i]
        print(droplist)
        self.data=self.data.drop(columns=droplist)
        
            
    def Label_Encoding(self):
        label_encoder = preprocessing.LabelEncoder() 
        x=pd.DataFrame(self.data.dtypes)
        ll=list(x[x[0]=="object"].index)
        for i in ll:
            self.data[i]= label_encoder.fit_transform(self.data[i]) 
    
    def StdScale(self):
        for i in self.data.columns:
            if self.data[i].dtypes!='object' and i!='Churn':
                scale = StandardScaler().fit(self.data[[i]])
    
                self.data[i] = scale.transform(self.data[[i]])

        
    ## DANGER ZONE Col Spare NEEDED To Keep y_pred. RAM HOGGING FUNCTION .
    #Use Wisely! Plus Parallize the operation for better efficacy? Maybe???
                
    def drop_vif(self,thresh=5,col_Spare=['Churn','intercept']):

        
        vif=Utils_Suite(self.data).compute_vif()
        z1=vif[vif["vif"]>thresh]
        z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
        while True:
            try:
                col=z1.iloc[0,0]
                if z1.empty:
                    break
                if col in col_Spare:
                    z1=z1.iloc[1:]
                    continue
                self.data=self.data.drop(col,axis=1)
                vif=Utils_Suite(self.data).compute_vif()
                z1=vif[vif["vif"]>thresh]
                z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
            except IndexError:
                break
        
    # Wrapper Function to dump data to a variable  
    def write_df(self):
        return self.data
    

In [10]:
x=PreProcess(data)
data=x.write_df()

customerID
gender
Partner
Dependents
PhoneService
MultipleLines
InternetService
OnlineSecurity
OnlineBackup
DeviceProtection
TechSupport
StreamingTV
StreamingMovies
Contract
PaperlessBilling
PaymentMethod
Churn
['customerID']
['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']


In [6]:
from sklearn.model_selection import train_test_split

In [11]:
x=data.iloc[:,:-1].values
y=data.iloc[:,-1].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2024)

### Support Vector Classifier

In [13]:
from sklearn import svm
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
#Create a svm Classifier
clf = svm.SVC(kernel='poly') # Linear Kernel

#Train the model using the training sets
clf.fit(x_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(x_test)
accuracy_score(y_test, y_pred)

0.8023696682464455

In [45]:
clf.get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

#### Grid SearchCV (SVC)

In [52]:
param={
    
    'kernel':['linear','poly','rbf'],
    'gamma':['scale','auto'],
    'shrinking':[True,False],
    
    'probability':[True,False],
    'break_ties':[True,False],
    


  }
cls_grid=svm.SVC()
gr=GridSearchCV(cls_grid,param_grid=param,cv=5)  # class model,param,crossval=5
gr.fit(x_train,y_train)
gr.best_params_

{'kernel': 'linear'}

### Random Search CV(SVC)

In [53]:
gr=RandomizedSearchCV(cls_grid,param_distributions=param,cv=5)  
gr.fit(x_train,y_train)
gr.best_params_

e:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'kernel': 'linear'}

### Decision Tree Classifier

In [14]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(criterion="entropy",max_depth=4)
tree.fit(x_train,y_train)
y_pred=tree.predict(x_test)
(y_test,y_pred)

confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

0.7909952606635071

### Grid Search CV (Decision Tree Classifier)

In [41]:
param={
    
    'criterion':['gini','entropy'],
    'splitter':['best','random'],
    
    'max_depth':[i for i in range(10,20,1)],
    'min_samples_split':[i for i in range(2,10)],
    'max_features':['sqrt','log2'],


  }
cls_grid=DecisionTreeClassifier()
gr=GridSearchCV(cls_grid,param_grid=param,cv=5)  # class model,param,crossval=5
gr.fit(x_train,y_train)
gr.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 7,
 'splitter': 'random'}

### Random SearchCV(Decision Tree)

In [42]:
gr=RandomizedSearchCV(cls_grid,param_distributions=param,cv=5)  
gr.fit(x_train,y_train)
gr.best_params_

{'splitter': 'random',
 'min_samples_split': 9,
 'max_features': 'sqrt',
 'max_depth': 11,
 'criterion': 'entropy'}

### Random Forest Classifier

In [15]:
from sklearn.preprocessing import StandardScaler


ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.transform(x_test)

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=75,criterion='gini',bootstrap=True,random_state=1101)
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)
accuracy_score(y_pred,y_test)

0.8023696682464455

#### GridSearchCV(RandomForest)

In [21]:
param={
    'n_estimators':[i for i in range(50,100,25)],
    'criterion':['gini','entropy'],
    'bootstrap':[True,False]

  }
cls_grid=RandomForestClassifier()
gr=GridSearchCV(cls_grid,param_grid=param,cv=5)  # class model,param,crossval=5
gr.fit(x_train,y_train)
gr.best_params_

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 75}

### Random SearchCV(Random Forest)

In [23]:
gr=RandomizedSearchCV(cls_grid,param_distributions=param,cv=5)  
gr.fit(x_train,y_train)
gr.best_params_

e:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 75, 'criterion': 'gini', 'bootstrap': True}

### Adaboost

In [33]:
from sklearn.ensemble import AdaBoostClassifier
cl=AdaBoostClassifier(n_estimators=25, algorithm="SAMME.R",learning_rate=1.0, random_state=0)
cl.fit(x_train,y_train)
y_pred=cl.predict(x_test)
accuracy_score(y_pred,y_test)

0.8118483412322275

#### GridSearchCV(AdaBoost)

In [32]:
param={
    'n_estimators':[i for i in range(0,100,25)],
    'learning_rate':[i for i in np.arange(0,10,0.5)],
    'algorithm':['SAMME','SAMME.R'],
  }
cls_grid=AdaBoostClassifier()
gr=GridSearchCV(cls_grid,param_grid=param,cv=5)  # class model,param,crossval=5
gr.fit(x_train,y_train)
gr.best_params_

e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:655: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:486: UserWarning: Sample weights have reached infinite values, at iteration 15, causing overflow. Iterations stopped. Try lowering the learning rate.
  return super().fit(X, y, sample_weight)
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:655: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:486: UserWarning: Sample weights have reached infinite values, at iteration 15, causing overflow. Iterations stopped. Try lowering the learning rate.
  return super().fit(X, y, sample_weight)
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:655: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_b

{'algorithm': 'SAMME.R', 'learning_rate': 1.0, 'n_estimators': 25}

#### RandomSearch CV(Adaboost)

In [34]:
gr=RandomizedSearchCV(cls_grid,param_distributions=param,cv=5)  
gr.fit(x_train,y_train)
gr.best_params_

e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:655: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:486: UserWarning: Sample weights have reached infinite values, at iteration 4, causing overflow. Iterations stopped. Try lowering the learning rate.
  return super().fit(X, y, sample_weight)
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:655: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:486: UserWarning: Sample weights have reached infinite values, at iteration 4, causing overflow. Iterations stopped. Try lowering the learning rate.
  return super().fit(X, y, sample_weight)
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boosting.py:655: RuntimeWarning: overflow encountered in exp
  sample_weight = np.exp(
e:\anaconda\lib\site-packages\sklearn\ensemble\_weight_boo

{'n_estimators': 25, 'learning_rate': 1.0, 'algorithm': 'SAMME.R'}